In [6]:
import os
import warnings

import DataReader

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

import warnings

warnings.filterwarnings(action='ignore')

# 주피터 노트북 환경에서 현재 작업 디렉토리 경로를 얻습니다.
current_dir_path = os.getcwd()

# info.yaml 파일의 경로를 현재 작업 디렉토리를 기반으로 구성합니다.
yaml_file_path = os.path.join(current_dir_path, 'info.yaml')
target_col = 'target'

ROOT_PATH = current_dir_path

train, test, label = DataReader.read_from_db(
    yaml_file_path=yaml_file_path,
    table_name="adult_income",
    label_col_name=target_col)


In [7]:
train_cat_columns = train.select_dtypes(include='object').columns
train_num_columns = train.select_dtypes(exclude='object').columns

print('Categorical Columns: \n{}\n\n Numeric Columns: \n{}\n'.format(train_cat_columns,train_num_columns))

Categorical Columns: 
Index(['workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country'],
      dtype='object')

 Numeric Columns: 
Index(['index', 'id', 'age', 'fnlwgt', 'education.num', 'capital.gain',
       'capital.loss', 'hours.per.week'],
      dtype='object')



age, education_num, hours.per.week 이 target과 상대적으로 큰 상관관계 보임.

In [8]:
train.columns = train.columns.str.replace('.','_')
test.columns = test.columns.str.replace('.','_')

In [9]:
train.drop('fnlwgt',axis=1,inplace=True)
test.drop('fnlwgt',axis=1,inplace=True)

### 결측값 처리

In [10]:

# 'workclass' 열의 결측값을 'Never-worked'로 대체
train['workclass'] = train['workclass'].apply(lambda x: 'Never-worked' if x is None else x)

# 'occupation' 열의 결측값을 'Others'로 대체
# 'occupation' 열에서 결측값이 의미 있는 값으로 간주될 수 있음을 반영
train['occupation'] = train['occupation'].apply(lambda x: 'Others' if x is None else x)

# 'native.country'와 race의 상관관계가 큼을 확인.
# 'native.country' 열의 결측값을 race에 따른 native.country의 분포를 참조하여 채운다.

# 각 'race'에 대한 'native.country'의 가장 흔한 값 찾기
most_common_countries_per_race = train.groupby('race')['native_country'].apply(lambda x: x.mode().iloc[0])

# 'native.country'의 결측값을 해당 'race'의 가장 흔한 값으로 채우기
for race, common_country in most_common_countries_per_race.items():
    condition = (train['race'] == race) & (train['native_country'].isnull())
    train.loc[condition, 'native_country'] = common_country

# 변경 사항 확인을 위한 출력
print(train[['workclass', 'occupation', 'native_country']].isnull().sum())

occupation_counts = train['occupation'].value_counts()
print(occupation_counts)

native_country_counts = train['native_country'].value_counts()
print(native_country_counts)


workclass         0
occupation        0
native_country    0
dtype: int64
occupation
Exec-managerial      2113
Craft-repair         2101
Prof-specialty       2085
Adm-clerical         1893
Others               1843
Sales                1829
Other-service        1677
Machine-op-inspct    1040
Transport-moving      785
Handlers-cleaners     695
Farming-fishing       508
Tech-support          475
Protective-serv       350
Priv-house-serv        83
Armed-Forces            3
Name: count, dtype: int64
native_country
United-States                 15976
Mexico                          355
Philippines                     108
Germany                          75
Canada                           63
Puerto-Rico                      59
El-Salvador                      58
Cuba                             49
India                            47
England                          46
Jamaica                          42
China                            40
South                            39
Italy            

#### Feature Engineering

In [11]:
cat_columns = train.select_dtypes(include='object').columns
num_columns = train.select_dtypes(exclude='object').columns

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train, label,
                                                      test_size=0.3,
                                                      shuffle=True,
                                                      stratify=label)

In [13]:
x_train = x_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
x_test = test

In [14]:
x_test['workclass'] = x_test['workclass'].apply(lambda x: 'Never-worked' if x is None else x)

# 'occupation' 열의 결측값을 'Others'로 대체
# 'occupation' 열에서 결측값이 의미 있는 값으로 간주될 수 있음을 반영
x_test['occupation'] = x_test['occupation'].apply(lambda x: 'Others' if x is None else x)

# 'native.country'와 race의 상관관계가 큼을 확인.
# 'native.country' 열의 결측값을 race에 따른 native.country의 분포를 참조하여 채운다.

# 각 'race'에 대한 'native.country'의 가장 흔한 값 찾기
most_common_countries_per_race = x_test.groupby('race')['native_country'].apply(lambda x: x.mode().iloc[0])

# 'native.country'의 결측값을 해당 'race'의 가장 흔한 값으로 채우기
for race, common_country in most_common_countries_per_race.items():
    condition = (x_test['race'] == race) & (x_test['native_country'].isnull())
    x_test.loc[condition, 'native_country'] = common_country

# 변경 사항 확인을 위한 출력
print(x_test[['workclass', 'occupation', 'native_country']].isnull().sum())

occupation_counts = x_test['occupation'].value_counts()
print(occupation_counts)

native_country_counts = x_test['native_country'].value_counts()
print(native_country_counts)

workclass         0
occupation        0
native_country    0
dtype: int64
occupation
Prof-specialty       2055
Craft-repair         1998
Exec-managerial      1953
Adm-clerical         1877
Sales                1821
Other-service        1618
Machine-op-inspct     962
Transport-moving      812
Handlers-cleaners     675
Farming-fishing       486
Tech-support          453
Protective-serv       299
Priv-house-serv        66
Armed-Forces            6
Name: count, dtype: int64
native_country
United-States                 13777
Mexico                          288
Philippines                      90
Germany                          62
Canada                           58
Puerto-Rico                      55
India                            53
El-Salvador                      48
Cuba                             46
England                          44
South                            41
Jamaica                          39
Italy                            35
China                            35
Vietnam

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train[num_columns] = scaler.fit_transform(x_train[num_columns])
x_valid[num_columns] = scaler.transform(x_valid[num_columns])
x_test[num_columns]  = scaler.transform(x_test[num_columns])

In [18]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

ohe = OneHotEncoder()

# ohe.fit(x_train[cat_columns])
ohe.fit(pd.concat([x_train[cat_columns], x_valid[cat_columns], x_test[cat_columns]]))
ohe_columns = ohe.get_feature_names_out()

new_x_train_cat = pd.DataFrame(ohe.transform(x_train[cat_columns]), columns=ohe_columns)
new_x_valid_cat = pd.DataFrame(ohe.transform(x_valid[cat_columns]), columns=ohe_columns)
new_x_test_cat  = pd.DataFrame(ohe.transform(x_test[cat_columns]), columns=ohe_columns)

ValueError: Shape of passed values is (12236, 1), indices imply (12236, 100)